In [97]:
import pandas as pd
import time
import re
from crontab import CronTab
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

url = 'https://bet.hkjc.com/football/odds/odds_inplay.aspx?lang=en'
year = '2020'

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(7) # let the potato rest for a while
games_info = driver.find_element_by_id('ActiveMatchesOdds')
soup = BeautifulSoup(games_info.get_attribute('innerHTML'), 'html.parser')
driver.quit()


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/TysonWu/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]


In [103]:
event = [link.text.strip() for link in soup.find_all("div", "cday alloddsLink")]
home = [link.text.strip() for link in soup.find_all("span", "teamname")][::2]
away = [link.text.strip() for link in soup.find_all("span", "teamname")][1::2]
time = [re.sub('Expected In Play start selling time: ', '' ,link.text.strip()) 
        for link in soup.find_all("div", "cesst")][1:]

games = pd.DataFrame({'event': event, 
                      'home': home, 
                      'away': away, 
                      'time': time})
games = games.iloc[1:,]
current_year = str(datetime.now().year)
games['time'] = games.time.apply(lambda x: datetime.strptime(
    current_year+'/'+x, '%Y/%d/%m %H:%M'))
games['month'] = games['time'].apply(lambda x: x.month)
games['day'] = games['time'].apply(lambda x: x.day)
games['hour'] = games['time'].apply(lambda x: x.hour)
games['minute'] = games['time'].apply(lambda x: x.minute)
# games['crontab_time'] = ' '.join(games.minute, games.hour, games.day, games.month)
games

,event,home,away,time,month,day,hour,minute
1,FRI 31,Monarcas Morelia,Tijuana,2020-02-15 11:00:00,2,15,11,0
2,SAT 1,Wellington Phoenix,Melbourne City,2020-02-15 14:00:00,2,15,14,0
3,SAT 2,Western Sydney Wanderers,Newcastle Jets,2020-02-15 16:30:00,2,15,16,30
4,SAT 3,Mallorca,Alaves,2020-02-15 20:00:00,2,15,20,0
5,SAT 7,Southampton,Burnley,2020-02-15 20:30:00,2,15,20,30
6,SAT 9,Lecce,SPAL,2020-02-15 22:00:00,2,15,22,0
7,SAT 12,RB Leipzig,Werder Bremen,2020-02-15 22:30:00,2,15,22,30
8,SAT 13,Union Berlin,Leverkusen,2020-02-15 22:30:00,2,15,22,30
9,SAT 17,Barcelona,Getafe,2020-02-15 23:00:00,2,15,23,0
10,SAT 46,Amiens,Paris Saint Germain,2020-02-16 00:30:00,2,16,0,30


In [99]:
cron = CronTab(user=True)